In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 15.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import ultralytics

In [ ]:
ultralytics.checks()

Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/107.7 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = YOLO("yolov8x.pt")

100%|██████████| 131M/131M [00:00<00:00, 304MB/s]


In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
import xml.etree.ElementTree as ET
import os

# Define the classes
classes = ["person"]

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    with open(f"{output_dir}/{root.find('filename').text.split('.')[0]}.txt", 'w') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id= classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w, h), b)
            out_file.write(f"{cls_id} " + " ".join([f"{a:.6f}" for a in bb]) + '\n')

xml_dir = r"/content/drive/My Drive/yolo/labels"
output_dir = r"/content/drive/My Drive/yolo/person/labels"

os.makedirs(output_dir, exist_ok=True)

for xml_file in os.listdir(xml_dir):
    if xml_file.endswith(".xml"):
        convert_annotation(os.path.join(xml_dir, xml_file), output_dir)


In [ ]:
model.train(
    data=r"/content/drive/My Drive/yolo/person_data.yaml",
    epochs=1,
    imgsz=640,
    batch=16,
    name='custom_yolov8'
)

Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/yolo/person_data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov84, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/drive/MyDrive/yolo/person/labels/train.cache... 322 images, 94 backgrounds, 0 corrupt: 100%|██████████| 416/416 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/yolo/person/labels/train.cache... 322 images, 94 backgrounds, 0 corrupt: 100%|██████████| 416/416 [00:00<?, ?it/s]

Plotting labels to runs/detect/custom_yolov84/labels.jpg... 


...

        1/1         0G     0.9754       1.53      1.144         58        640: 100%|██████████| 26/26 [06:50<00:00, 15.78s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [02:16<00:00, 10.49s/it]


                   all        416       1107      0.897      0.525      0.758      0.517

1 epochs completed in 0.155 hours.
Optimizer stripped from runs/detect/custom_yolov84/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/custom_yolov84/weights/best.pt, 6.2MB

Validating runs/detect/custom_yolov84/weights/best.pt...
Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [02:01<00:00,  9.36s/it]


                   all        416       1107      0.897      0.526      0.758      0.517
Speed: 8.7ms preprocess, 258.9ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to runs/detect/custom_yolov84


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fddafb38970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
!pip install pyyaml


In [ ]:
import yaml

# Path to your YAML file in Google Drive
yaml_file = "/content/drive/My Drive/yolo/person_data.yaml"

# Open and read the YAML file
with open(yaml_file, 'r') as file:
    data = yaml.safe_load(file)

# Print the contents of the YAML file
print(data)


{'path': '/content/drive/MyDrive/yolo/person', 'train': {'images': '/content/drive/MyDrive/yolo/person/images/train', 'labels': '/content/drive/MyDrive/yolo/person/labels/train'}, 'val': {'images': '/content/drive/MyDrive/yolo/person/images/train', 'labels': '/content/drive/MyDrive/yolo/person/labels/train'}, 'nc': 1, 'names': ['person'], 'weights': 'yolov8n.pt'}


In [ ]:
# yaml to yolo format

import os
from PIL import Image
import xml.etree.ElementTree as ET

class_mapping = {
                'hard-hat': 0,
                'gloves': 1,
                'boots': 2,
                'vest': 3,
                'ppe-suit': 4
                }

def convert_to_yolo_format(bbox, img_width, img_height):
    x_center = (bbox['xmin'] + bbox['xmax']) / (2 * img_width)
    y_center = (bbox['ymin'] + bbox['ymax']) / (2 * img_height)
    width = (bbox['xmax'] - bbox['xmin']) / img_width
    height = (bbox['ymax'] - bbox['ymin']) / img_height
    return x_center, y_center, width, height

def crop_and_adjust_annotations(image_path, annotation, person_bbox):

    image = Image.open(image_path)


    cropped_image = image.crop((person_bbox['xmin'], person_bbox['ymin'], person_bbox['xmax'], person_bbox['ymax']))


    crop_width = person_bbox['xmax'] - person_bbox['xmin']
    crop_height = person_bbox['ymax'] - person_bbox['ymin']


    adjusted_objects = []
    for obj in annotation['object']:
        if obj['name'] != 'person' and obj['name'] in class_mapping:
            bbox = obj['bndbox']

            if (bbox['xmin'] >= person_bbox['xmin'] and bbox['xmax'] <= person_bbox['xmax'] and
                bbox['ymin'] >= person_bbox['ymin'] and bbox['ymax'] <= person_bbox['ymax']):

                adjusted_bbox = {
                    'xmin': bbox['xmin'] - person_bbox['xmin'],
                    'ymin': bbox['ymin'] - person_bbox['ymin'],
                    'xmax': bbox['xmax'] - person_bbox['xmin'],
                    'ymax': bbox['ymax'] - person_bbox['ymin']
                }

                x_center, y_center, width, height = convert_to_yolo_format(adjusted_bbox, crop_width, crop_height)
                class_id = class_mapping[obj['name']]
                adjusted_objects.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return cropped_image, adjusted_objects

def process_annotation(annotation_path, image_folder, output_folder):

    tree = ET.parse(annotation_path)
    root = tree.getroot()


    filename = root.find('filename').text


    annotation = {
        'filename': filename,
        'object': []
    }

    for obj in root.findall('object'):
        name = obj.find('name').text
        bbox = obj.find('bndbox')
        annotation['object'].append({
            'name': name,
            'bndbox': {
                'xmin': int(bbox.find('xmin').text),
                'ymin': int(bbox.find('ymin').text),
                'xmax': int(bbox.find('xmax').text),
                'ymax': int(bbox.find('ymax').text)
            }
        })


    person_count = 0
    for obj in annotation['object']:
        if obj['name'] == 'person':
            person_count += 1
            person_bbox = obj['bndbox']


            image_path = os.path.join(image_folder, filename)
            cropped_image, adjusted_objects = crop_and_adjust_annotations(image_path, annotation, person_bbox)


            output_image_name = f"{os.path.splitext(filename)[0]}_person{person_count}.jpg"
            output_image_path = os.path.join(output_folder, 'images', output_image_name)
            cropped_image.save(output_image_path)


            output_annotation_path = os.path.join(output_folder, 'labels', f"{os.path.splitext(output_image_name)[0]}.txt")
            with open(output_annotation_path, 'w') as f:
                f.write('\n'.join(adjusted_objects))

def main():
    input_folder = '/content/drive/My Drive/yolo/person'
    output_folder = '/content/drive/My Drive/yolo/ppe/images'
    image_folder = os.path.join(input_folder, 'images')
    annotation_folder = os.path.join(input_folder, 'labels')


    os.makedirs(os.path.join(output_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'labels'), exist_ok=True)


    for annotation_file in os.listdir(annotation_folder):
        if annotation_file.endswith('.xml'):
            annotation_path = os.path.join(annotation_folder, annotation_file)
            process_annotation(annotation_path, image_folder, output_folder)

if __name__ == '__main__':
    main()

In [ ]:
model.train(
    data='/content/drive/My Drive/yolo/ppe_data.yaml',   # Path to the YAML configuration file
    epochs=1,          # Number of epochs to train
    imgsz=640,          # Image size for training
    batch=64,           # Batch size
    name='ppe_yolov8' # Name of the training run
)

New https://pypi.org/project/ultralytics/8.2.81 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.31 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/ppe_data.yml, epochs=1, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=ppe_yolov82, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

train: Scanning /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/labels/train.cache... 1281 images, 210 backgrounds, 3 corrupt: 100%|██████████| 1284/1284 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/images/train/001305_person10.jpg: ignoring corrupt image/label: image size (21, 7) <10 pixels
train: WARNING ⚠️ /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/images/train/001790_person5.jpg: ignoring corrupt image/label: image size (18, 7) <10 pixels
train: WARNING ⚠️ /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/images/train/001790_person6.jpg: ignoring corrupt image/label: image size (16, 7) <10 pixels



val: Scanning /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/labels/train.cache... 1281 images, 210 backgrounds, 3 corrupt: 100%|██████████| 1284/1284 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/images/train/001305_person10.jpg: ignoring corrupt image/label: image size (21, 7) <10 pixels
train: WARNING ⚠️ /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/images/train/001790_person5.jpg: ignoring corrupt image/label: image size (18, 7) <10 pixels
train: WARNING ⚠️ /Users/cilvosimon/Codes/YOLOv8_PPE/ppe_dataset2/images/train/001790_person6.jpg: ignoring corrupt image/label: image size (16, 7) <10 pixels
Plotting labels to runs/detect/ppe_yolov82/labels.jpg... 


...

        1/1         0G      1.425       4.04      1.422          3        640: 100%|██████████| 21/21 [11:27<00:00, 32.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [02:00<00:00, 10.99s/it]


                   all       1281       2483     0.0128      0.573       0.14     0.0997

1 epochs completed in 0.226 hours.
Optimizer stripped from runs/detect/ppe_yolov82/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/ppe_yolov82/weights/best.pt, 6.2MB

Validating runs/detect/ppe_yolov82/weights/best.pt...
Ultralytics YOLOv8.1.31 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M1)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [01:46<00:00,  9.66s/it]


                   all       1281       2483     0.0127      0.574       0.14     0.0997
              hard-hat       1281        850     0.0465      0.962      0.672      0.454
                gloves       1281        440    0.00109      0.145    0.00139   0.000672
                  mask       1281          9   2.33e-05      0.222      0.115      0.115
                 boots       1281        843    0.00586      0.255    0.00414    0.00168
                  vest       1281        169     0.0112      0.899     0.0233     0.0111
              ppe-suit       1281        172     0.0118      0.959     0.0258     0.0157
Speed: 0.5ms preprocess, 77.3ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/ppe_yolov82


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x35bcd3550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [ ]:
import yaml

# Path to your YAML file in Google Drive
yaml_file = "/content/drive/My Drive/yolo/ppe_data.yaml"

# Open and read the YAML file
with open(yaml_file, 'r') as file:
    data = yaml.safe_load(file)

# Print the contents of the YAML file
print(data)


{'path': '/content/drive/MyDrive/yolo/ppe', 'train': {'images': '/content/drive/MyDrive/yolo/ppe/images', 'labels': '/content/drive/MyDrive/yolo/ppe/labels'}, 'val': {'images': '/content/drive/MyDrive/yolo/ppe/images', 'labels': '/content/drive/MyDrive/yolo/ppe/labels'}, 'nc': 5, 'names': ['hard-hat', 'gloves', 'boots', 'vest', 'ppe-suit'], 'weights': 'yolov8m.pt'}


In [ ]:
model.train(
    data='/content/drive/My Drive/yolo/ppe_data.yaml',   # Path to the YAML configuration file
    epochs=1,          # Number of epochs to train
    imgsz=640,          # Image size for training
    batch=64,           # Batch size
    name='ppe_yolov8' # Name of the training run
)

Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/yolo/ppe_data.yaml, epochs=1, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=ppe_yolov84, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

/usr/local/lib/python3.10/dist-packages/ultralytics/engine/trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /content/drive/MyDrive/yolo/ppe/labels.cache... 1281 images, 210 backgrounds, 3 corrupt: 100%|██████████| 1284/1284 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/yolo/ppe/images/001305_person10.jpg: ignoring corrupt image/label: image size (21, 7) <10 pixels
train: WARNING ⚠️ /content/drive/MyDrive/yolo/ppe/images/001790_person5.jpg: ignoring corrupt image/label: image size (18, 7) <10 pixels
train: WARNING ⚠️ /content/drive/MyDrive/yolo/ppe/images/001790_person6.jpg: ignoring corrupt image/label: image size (16, 7) <10 pixels


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/yolo/ppe/labels.cache... 1281 images, 210 backgrounds, 3 corrupt: 100%|██████████| 1284/1284 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/yolo/ppe/images/001305_person10.jpg: ignoring corrupt image/label: image size (21, 7) <10 pixels
train: WARNING ⚠️ /content/drive/MyDrive/yolo/ppe/images/001790_person5.jpg: ignoring corrupt image/label: image size (18, 7) <10 pixels
train: WARNING ⚠️ /content/drive/MyDrive/yolo/ppe/images/001790_person6.jpg: ignoring corrupt image/label: image size (16, 7) <10 pixels


Plotting labels to runs/detect/ppe_yolov84/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/ppe_yolov84
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

def load_models():
    person_model = YOLO('/content/drive/My Drive/yolo/runs/detect/custom_yolov82/weights/best.pt')
    ppe_model = YOLO('/Users/cilvosimon/Codes/YOLOv8_PPE/runs/detect/ppe_yolov82/weights/best.pt')
    return person_model, ppe_model

def perform_inference(image_path, person_model, ppe_model):
    # Read the image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Perform person detection
    person_results = person_model(image)[0]

    # Process each detected person
    for person_box in person_results.boxes.xyxy:
        x1, y1, x2, y2 = map(int, person_box[:4])

        # Crop the person from the image
        person_crop = image[y1:y2, x1:x2]

        # Perform PPE detection on the cropped image
        ppe_results = ppe_model(person_crop)[0]

        # Convert PPE detections back to full image coordinates
        for ppe_box in ppe_results.boxes.xyxy:
            ppe_x1, ppe_y1, ppe_x2, ppe_y2 = map(int, ppe_box[:4])
            full_ppe_x1 = x1 + ppe_x1
            full_ppe_y1 = y1 + ppe_y1
            full_ppe_x2 = x1 + ppe_x2
            full_ppe_y2 = y1 + ppe_y2

            # Draw PPE bounding box
            cv2.rectangle(original_image, (full_ppe_x1, full_ppe_y1), (full_ppe_x2, full_ppe_y2), (0, 255, 0), 2)

            # Add label and confidence
            label = ppe_results.names[int(ppe_box.cls)]
            conf = float(ppe_box.conf)
            label_text = f"{label}: {conf:.2f}"
            cv2.putText(original_image, label_text, (full_ppe_x1, full_ppe_y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw person bounding box
        cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(original_image, "Person", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return original_image

def process_directory(input_dir, output_dir, person_model, ppe_model):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"inference_{filename}")

            result_image = perform_inference(input_path, person_model, ppe_model)
            cv2.imwrite(output_path, result_image)
            print(f"Processed: {filename}")

def main():
    input_directory = '/Users/cilvosimon/Codes/YOLOv8_PPE/datasets'
    output_directory = '/Users/cilvosimon/Codes/YOLOv8_PPE/test'

    person_model, ppe_model = load_models()
    process_directory(input_directory, output_directory, person_model, ppe_model)

if __name__ == '__main__':
    main()


0: 640x192 (no detections), 48.0ms
Speed: 5.1ms preprocess, 48.0ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 192)
Processed: cropped_1.jpg

0: 640x224 (no detections), 33.9ms
Speed: 0.8ms preprocess, 33.9ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 224)
Processed: cropped_3.jpg

0: 640x224 (no detections), 32.5ms
Speed: 0.9ms preprocess, 32.5ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 224)
Processed: cropped_2.jpg

0: 640x640 (no detections), 85.8ms
Speed: 1.5ms preprocess, 85.8ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Processed: whole_image.jpg


tryyyy


In [ ]:
!mkdir -p /content/drive/My\ Drive/yolo/weights


In [ ]:
!mv /content/drive/My\ Drive/yolo/person_yolov8x2x/person_yolov8x2/weights/best.pt /content/drive/My\ Drive/yolo/weights/
!mv /content/drive/My\ Drive/yolo/ppe_model.pt /content/drive/My\ Drive/yolo/weights/


mv: cannot stat '/content/drive/My Drive/yolo/ppe_model.pt': No such file or directory


In [ ]:
person_model_path = '/content/drive/My Drive/yolo/weights/best.pt'
ppe_model_path = '/content/drive/My Drive/yolo/weights/ppe_model.pt'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify the paths to ensure files are in the correct locations
!ls /content/drive/My\ Drive/yolo

# Install necessary packages
!pip install ultralytics

# Run the inference script
!python /content/drive/My\ Drive/yolo/inference.py \
    /content/drive/My\ Drive/yolo/images \
    /content/drive/My\ Drive/yolo/test \
    /content/drive/My\ Drive/yolo/weights/best.pt \
    /content/drive/My\ Drive/yolo/weights/ppe_model.pt


Streaming output truncated to the last 5000 lines.
0: 640x448 3 persons, 3255.6ms
Speed: 4.1ms preprocess, 3255.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x256 1 hard-hat, 790.1ms
Speed: 1.5ms preprocess, 790.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 256)

0: 640x448 1 hard-hat, 1 gloves, 1692.7ms
Speed: 3.3ms preprocess, 1692.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 448)

0: 640x224 1 hard-hat, 912.0ms
Speed: 2.0ms preprocess, 912.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 224)
Processed: 001059.jpg

0: 544x640 3 persons, 4776.5ms
Speed: 4.1ms preprocess, 4776.5ms inference, 1.1ms postprocess per image at shape (1, 3, 544, 640)

0: 640x320 1 hard-hat, 809.2ms
Speed: 1.8ms preprocess, 809.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 320)

0: 640x256 (no detections), 659.4ms
Speed: 2.0ms preprocess, 659.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 256)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify the paths to ensure files are in the correct locations
!ls /content/drive/My\ Drive/yolo

# Install necessary packages
!pip install ultralytics

# Run the inference script
!python /content/drive/My\ Drive/yolo/inference.py \
    /content/drive/My\ Drive/yolo/images \
    /content/drive/My\ Drive/yolo/test \
    /content/drive/My\ Drive/yolo/weights/person_detect.pt \
    /content/drive/My\ Drive/yolo/weights/ppe_model.pt


Streaming output truncated to the last 5000 lines.
Speed: 2.6ms preprocess, 1505.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2390-_png_jpg.rf.fa4cf091a0bc051c044e2505719d3971.jpg

0: 640x640 (no detections), 1575.2ms
Speed: 2.7ms preprocess, 1575.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: 001246_jpg.rf.05724a1c67f05c4fbd6fb3d872bc98b4.jpg

0: 640x640 (no detections), 1578.8ms
Speed: 3.4ms preprocess, 1578.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: ppe_1166_jpg.rf.ccad8bba387168ec13aa4f3ce7848f27.jpg

0: 640x640 (no detections), 1768.7ms
Speed: 3.4ms preprocess, 1768.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Processed: 02567_jpg.rf.d1327d7fe3826f7ae0976fc5e90f94aa.jpg

0: 640x640 1 person, 2452.2ms
Speed: 4.0ms preprocess, 2452.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x64 (no detections), 273.9ms
Speed: 1.0ms preproc

In [ ]:
!python /content/drive/My\ Drive/yolo/inference.py \
    /content/drive/My\ Drive/yolo/images \
    /content/drive/My\ Drive/yolo/test \
    /content/drive/My\ Drive/yolo/person_detect.pt \
    /content/drive/My\ Drive/yolo/ppe_model.pt


In [ ]:
import os
import cv2
import argparse
import numpy as np
from ultralytics import YOLO

def load_models(person_model_path, ppe_model_path):
    person_model = YOLO('/content/drive/My Drive/yolo/person_detec.pt')
    ppe_model = YOLO('/content/drive/My Drive/yolo/ppe_model.pt')
    return person_model, ppe_model

def perform_inference(image_path, person_model, ppe_model):
    # Read the image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Perform person detection
    person_results = person_model(image)[0]

    # Process each detected person
    for person_box in person_results.boxes:
        # Extract coordinates and convert to integers
        person_box_xyxy = person_box.xyxy.numpy()  # Convert to numpy array
        for box in person_box_xyxy:
            x1, y1, x2, y2 = map(int, box[:4])

            # Crop the person from the image
            person_crop = image[y1:y2, x1:x2]

            # Perform PPE detection on the cropped image
            ppe_results = ppe_model(person_crop)[0]

            # Process each PPE detection
            for ppe_box in ppe_results.boxes:
                # Extract coordinates and convert to integers
                ppe_box_xyxy = ppe_box.xyxy.numpy()  # Convert to numpy array
                for box in ppe_box_xyxy:
                    ppe_x1, ppe_y1, ppe_x2, ppe_y2 = map(int, box[:4])

                    # Convert PPE detections back to full image coordinates
                    full_ppe_x1 = x1 + ppe_x1
                    full_ppe_y1 = y1 + ppe_y1
                    full_ppe_x2 = x1 + ppe_x2
                    full_ppe_y2 = y1 + ppe_y2

                    # Draw PPE bounding box
                    cv2.rectangle(original_image, (full_ppe_x1, full_ppe_y1), (full_ppe_x2, full_ppe_y2), (0, 255, 0), 2)

                    # Add label and confidence
                    label = ppe_results.names[int(box[5])]  # Class label
                    conf = float(box[4])  # Confidence
                    label_text = f"{label}: {conf:.2f}"
                    cv2.putText(original_image, label_text, (full_ppe_x1, full_ppe_y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw person bounding box
            cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(original_image, "Person", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return original_image

def process_directory(input_dir, output_dir, person_model, ppe_model):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"inference_{filename}")

            result_image = perform_inference(input_path, person_model, ppe_model)
            cv2.imwrite(output_path, result_image)
            print(f"Processed: {filename}")

def main(input_dir, output_dir, person_model_path, ppe_model_path):
    # Load models
    person_model, ppe_model = load_models(person_model_path, ppe_model_path)

    # Process images
    process_directory(input_dir, output_dir, person_model, ppe_model)


In [ ]:
import os

# Define paths
input_directory = '/content/drive/My Drive/yolo/images'
output_directory = '/content/drive/My Drive/yolo/test'
person_model_path = '/content/drive/My Drive/yolo/person_detect.pt'
ppe_model_path = '/content/drive/My Drive/yolo/ppe_model.pt'

# Ensure directories exist
os.makedirs(input_directory, exist_ok=True)
os.makedirs(output_directory, exist_ok=True)

# Run the script
!python inference.py {input_directory} {output_directory} {person_model_path} {ppe_model_path}


python3: can't open file '/content/inference.py': [Errno 2] No such file or directory


In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

def load_models():
    person_model = YOLO('/content/drive/My Drive/yolo/person_detec.pt')
    ppe_model = YOLO('/content/drive/My Drive/yolo/ppe_model.pt')
    return person_model, ppe_model

def perform_inference(image_path, person_model, ppe_model):
    # Read the image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Perform person detection
    person_results = person_model(image)[0]

    # Process each detected person
    for person_box in person_results.boxes:
        # Extract coordinates and convert to integers
        person_box_xyxy = person_box.xyxy.numpy()  # Convert to numpy array
        for box in person_box_xyxy:
            x1, y1, x2, y2 = map(int, box[:4])

            # Crop the person from the image
            person_crop = image[y1:y2, x1:x2]

            # Perform PPE detection on the cropped image
            ppe_results = ppe_model(person_crop)[0]

            # Process each PPE detection
            for ppe_box in ppe_results.boxes:
                # Extract coordinates and convert to integers
                ppe_box_xyxy = ppe_box.xyxy.numpy()  # Convert to numpy array
                for box in ppe_box_xyxy:
                    ppe_x1, ppe_y1, ppe_x2, ppe_y2 = map(int, box[:4])

                    # Convert PPE detections back to full image coordinates
                    full_ppe_x1 = x1 + ppe_x1
                    full_ppe_y1 = y1 + ppe_y1
                    full_ppe_x2 = x1 + ppe_x2
                    full_ppe_y2 = y1 + ppe_y2

                    # Draw PPE bounding box
                    cv2.rectangle(original_image, (full_ppe_x1, full_ppe_y1), (full_ppe_x2, full_ppe_y2), (0, 255, 0), 2)

                    # Add label and confidence
                    label = ppe_results.names[int(box[5])]  # Class label
                    conf = float(box[4])  # Confidence
                    label_text = f"{label}: {conf:.2f}"
                    cv2.putText(original_image, label_text, (full_ppe_x1, full_ppe_y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw person bounding box
            cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(original_image, "Person", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return original_image



def main():
    input_directory = '/content/drive/My Drive/yolo/images'
    output_directory ='/content/drive/My Drive/yolo/test'

    person_model, ppe_model = load_models()
    process_directory(input_directory, output_directory, person_model, ppe_model)

if __name__ == '__main__':
    main()


0: 640x640 (no detections), 1583.2ms
Speed: 4.5ms preprocess, 1583.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1571-_png_jpg.rf.ccf919d5ea025ddc24cf0a707b331249.jpg

0: 640x640 (no detections), 1581.2ms
Speed: 4.0ms preprocess, 1581.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2338-_png_jpg.rf.e79ac380146af51e53738fd6184e1808.jpg

0: 640x640 (no detections), 1608.8ms
Speed: 3.7ms preprocess, 1608.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1817-_png_jpg.rf.0c0c9d7ee4b875c6ad49937fc72182f6.jpg

0: 640x640 (no detections), 2395.4ms
Speed: 3.9ms preprocess, 2395.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec.jpg

0: 640x640 (no detections), 1880.0ms
Speed: 4.6ms preprocess, 1880.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2180-_png_jpg.rf.9d63bb305e7747d22fe9a1

IndexError: index 5 is out of bounds for axis 0 with size 4

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

def load_models(person_model_path, ppe_model_path):
    person_model = YOLO(person_model_path)
    ppe_model = YOLO(ppe_model_path)
    return person_model, ppe_model

def perform_inference(image_path, person_model, ppe_model):
    # Read the image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Perform person detection
    person_results = person_model(image)[0]

    # Convert tensor results to numpy arrays
    person_boxes = person_results.boxes.xyxy.cpu().numpy()
    person_confidences = person_results.boxes.conf.cpu().numpy()
    person_class_ids = person_results.boxes.cls.cpu().numpy().astype(int)

    # Process each detected person
    for i, person_box in enumerate(person_boxes):
        x1, y1, x2, y2 = map(int, person_box[:4])
        conf = float(person_confidences[i])
        label = "Person"

        # Draw person bounding box
        cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(original_image, f"{label}: {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Crop the person from the image
        person_crop = image[y1:y2, x1:x2]

        # Perform PPE detection on the cropped image
        ppe_results = ppe_model(person_crop)[0]

        # Convert PPE tensor results to numpy arrays
        ppe_boxes = ppe_results.boxes.xyxy.cpu().numpy()
        ppe_confidences = ppe_results.boxes.conf.cpu().numpy()
        ppe_class_ids = ppe_results.boxes.cls.cpu().numpy().astype(int)

        # Convert PPE detections back to full image coordinates
        for j, ppe_box in enumerate(ppe_boxes):
            ppe_x1, ppe_y1, ppe_x2, ppe_y2 = map(int, ppe_box[:4])
            full_ppe_x1 = x1 + ppe_x1
            full_ppe_y1 = y1 + ppe_y1
            full_ppe_x2 = x1 + ppe_x2
            full_ppe_y2 = y1 + ppe_y2

            # Draw PPE bounding box
            cv2.rectangle(original_image, (full_ppe_x1, full_ppe_y1), (full_ppe_x2, full_ppe_y2), (0, 255, 0), 2)

            # Add label and confidence
            label = ppe_results.names[ppe_class_ids[j]]
            conf = float(ppe_confidences[j])
            label_text = f"{label}: {conf:.2f}"
            cv2.putText(original_image, label_text, (full_ppe_x1, full_ppe_y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return original_image

def process_directory(input_dir, output_dir, person_model_path, ppe_model_path):
    os.makedirs(output_dir, exist_ok=True)

    person_model, ppe_model = load_models(person_model_path, ppe_model_path)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"inference_{filename}")

            result_image = perform_inference(input_path, person_model, ppe_model)
            cv2.imwrite(output_path, result_image)
            print(f"Processed: {filename}")

# Example usage:
input_directory = '/content/drive/My Drive/yolo/images'
output_directory = '/content/drive/My Drive/yolo/test'
person_model_path = '/content/drive/My Drive/yolo/runs/person_yolo8/weights/best.pt'
ppe_model_path =  '/content/drive/My Drive/yolo/runs/ppe_detect_yolov83/weights/best.pt'

process_directory(input_directory, output_directory, person_model_path, ppe_model_path)

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

def load_models(person_model_path, ppe_model_path):
    person_model = YOLO(person_model_path)
    ppe_model = YOLO(ppe_model_path)
    return person_model, ppe_model

def perform_inference(image_path, person_model, ppe_model):
    # Read the image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Perform person detection
    person_results = person_model(image)[0]

    # Process each detected person
    for person_box in person_results.boxes:
        # Convert tensor to numpy array
        person_box = person_box.cpu().numpy()

        # Extract coordinates and convert to integers
        x1, y1, x2, y2 = person_box[:4].astype(int) # Removed [0] and added .cpu().numpy()

        # Crop the person from the image
        person_crop = image[y1:y2, x1:x2]

        # Perform PPE detection on the cropped image
        ppe_results = ppe_model(person_crop)[0]

        # Process each PPE detection
        for ppe_box in ppe_results.boxes:
            # Convert tensor to numpy array
            ppe_box = ppe_box.cpu().numpy()

            # Extract coordinates and convert to integers
            ppe_x1, ppe_y1, ppe_x2, ppe_y2 = ppe_box[:4].astype(int) # Removed [0]

            # Convert PPE detections back to full image coordinates
            full_ppe_x1 = x1 + ppe_x1
            full_ppe_y1 = y1 + ppe_y1
            full_ppe_x2 = x1 + ppe_x2
            full_ppe_y2 = y1 + ppe_y2

            # Draw PPE bounding box
            cv2.rectangle(original_image, (full_ppe_x1, full_ppe_y1), (full_ppe_x2, full_ppe_y2), (0, 255, 0), 2)

            # Add label and confidence
            label = ppe_results.names[int(ppe_box[5])]  # Adjusted index for label
            conf = float(ppe_box[4])  # Confidence is typically at index 4
            label_text = f"{label}: {conf:.2f}"
            cv2.putText(original_image, label_text, (full_ppe_x1, full_ppe_y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw person bounding box
        cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(original_image, "Person", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return original_image

def process_directory(input_dir, output_dir, person_model, ppe_model):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"inference_{filename}")

            result_image = perform_inference(input_path, person_model, ppe_model)
            cv2.imwrite(output_path, result_image)
            print(f"Processed: {filename}")

def main(input_directory, output_directory, person_model_path, ppe_model_path):
    # Load models
    person_model, ppe_model = load_models(person_model_path, ppe_model_path)

    # Process images
    process_directory(input_directory, output_directory, person_model, ppe_model)

if __name__ == '__main__':
    # Set your paths here
    input_directory = '/content/drive/My Drive/yolo/images'
    output_directory = '/content/drive/My Drive/yolo/test'
    os.makedirs(output_directory, exist_ok=True)
    person_model_path = '/content/drive/My Drive/yolo/person_detect.pt'
    ppe_model_path = '/content/drive/My Drive/yolo/ppe_model.pt'

    # Call main with paths
    main(input_directory, output_directory, person_model_path, ppe_model_path)



0: 640x640 (no detections), 1563.2ms
Speed: 3.8ms preprocess, 1563.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1571-_png_jpg.rf.ccf919d5ea025ddc24cf0a707b331249.jpg

0: 640x640 (no detections), 1770.1ms
Speed: 5.4ms preprocess, 1770.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2338-_png_jpg.rf.e79ac380146af51e53738fd6184e1808.jpg

0: 640x640 (no detections), 2361.9ms
Speed: 3.5ms preprocess, 2361.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1817-_png_jpg.rf.0c0c9d7ee4b875c6ad49937fc72182f6.jpg

0: 640x640 (no detections), 2269.1ms
Speed: 4.0ms preprocess, 2269.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec.jpg

0: 640x640 (no detections), 2323.0ms
Speed: 7.2ms preprocess, 2323.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2180-_png_jpg.rf.9d63bb305e7747d22fe9a1

AttributeError: 'Boxes' object has no attribute 'astype'. See valid attributes below.

    A class for managing and manipulating detection boxes.

    This class provides functionality for handling detection boxes, including their coordinates, confidence scores,
    class labels, and optional tracking IDs. It supports various box formats and offers methods for easy manipulation
    and conversion between different coordinate systems.

    Attributes:
        data (torch.Tensor | numpy.ndarray): The raw tensor containing detection boxes and associated data.
        orig_shape (Tuple[int, int]): The original image dimensions (height, width).
        is_track (bool): Indicates whether tracking IDs are included in the box data.
        xyxy (torch.Tensor | numpy.ndarray): Boxes in [x1, y1, x2, y2] format.
        conf (torch.Tensor | numpy.ndarray): Confidence scores for each box.
        cls (torch.Tensor | numpy.ndarray): Class labels for each box.
        id (torch.Tensor | numpy.ndarra

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

def load_models():
    person_model = YOLO('/content/drive/My Drive/yolo/runs/person_yolo8/weights/best.pt')
    ppe_model = YOLO('/content/drive/My Drive/yolo/runs/ppe_detect_yolov83/weights/best.pt')
    return person_model, ppe_model

def perform_inference(image_path, person_model, ppe_model):
    # Read the image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Perform person detection
    person_results = person_model(image)[0]

    # Process each detected person
    for person_box in person_results.boxes.xyxy:
        x1, y1, x2, y2 = map(int, person_box[:4])

        # Crop the person from the image
        person_crop = image[y1:y2, x1:x2]

        # Perform PPE detection on the cropped image
        ppe_results = ppe_model(person_crop)[0]

        # Convert PPE detections back to full image coordinates
        for ppe_box in ppe_results.boxes.xyxy:
            ppe_x1, ppe_y1, ppe_x2, ppe_y2 = map(int, ppe_box[:4])
            full_ppe_x1 = x1 + ppe_x1
            full_ppe_y1 = y1 + ppe_y1
            full_ppe_x2 = x1 + ppe_x2
            full_ppe_y2 = y1 + ppe_y2

            # Draw PPE bounding box
            cv2.rectangle(original_image, (full_ppe_x1, full_ppe_y1), (full_ppe_x2, full_ppe_y2), (0, 255, 0), 2)

            # Add label and confidence
            label = ppe_results.names[int(ppe_box.cls)]
            conf = float(ppe_box.conf)
            label_text = f"{label}: {conf:.2f}"
            cv2.putText(original_image, label_text, (full_ppe_x1, full_ppe_y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw person bounding box
        cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(original_image, "Person", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return original_image

def process_directory(input_dir, output_dir, person_model, ppe_model):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"inference_{filename}")

            result_image = perform_inference(input_path, person_model, ppe_model)
            cv2.imwrite(output_path, result_image)
            print(f"Processed: {filename}")

def main():
    input_directory = '/content/drive/My Drive/yolo/images'
    output_directory = '/content/drive/My Drive/yolo/test'
    os.makedirs(output_directory, exist_ok=True)

    person_model, ppe_model = load_models()
    process_directory(input_directory, output_directory, person_model, ppe_model)

if __name__ == '__main__':
    main()


0: 640x640 (no detections), 1900.9ms
Speed: 12.9ms preprocess, 1900.9ms inference, 16.7ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1571-_png_jpg.rf.ccf919d5ea025ddc24cf0a707b331249.jpg

0: 640x640 (no detections), 1652.8ms
Speed: 4.8ms preprocess, 1652.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2338-_png_jpg.rf.e79ac380146af51e53738fd6184e1808.jpg

0: 640x640 (no detections), 1781.3ms
Speed: 4.4ms preprocess, 1781.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1817-_png_jpg.rf.0c0c9d7ee4b875c6ad49937fc72182f6.jpg

0: 640x640 (no detections), 2235.0ms
Speed: 14.1ms preprocess, 2235.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
Processed: -1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec.jpg

0: 640x640 (no detections), 7600.3ms
Speed: 13.4ms preprocess, 7600.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Processed: -2180-_png_jpg.rf.9d63bb305e7747d22f

AttributeError: 'Tensor' object has no attribute 'cls'